In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, HTML
import gpytorch
import torch
from gpytorch.kernels import Kernel
from numpy import ndarray
import os
import math
from math import floor


In [46]:
del shape

In [88]:
class GenSquaredExponetialKernel(Kernel):
    r"""
      Computes a covariance matrix based on the generalized squared exponential kernel
        between inputs :math:`\mathbf{x_1}` and :math:`\mathbf{x_2}`:
        .. math::
          \begin{equation*}
              k_{(\mathbf{x_1}, \mathbf{x_2}) = \sigma(x_1) sigma(x_2) \sqrt \left(\fract{2 l(x_1) l(x_2)}{l(x_1)^2 + l(x_2)^2} \right)
                \exp \left( - \fract{(x_1-x_2)^2}{l(x_1)^2 + l(x_2)^2} \right)  
          \end{equation*}
      where
      
      * :math:`\sigma` is the signal variance
        :math:`x_1` and :math:`x_2` scaled by the :attr:`lengthscale` parameter :math:`l`.
    """
    has_lengthscale = False

    def __init__(self, s1, s2, l1 , l2, log_transformed = False, **kwargs):
        
        if isinstance(s1, ndarray):
            self.s1 = torch.tensor(s1, dtype=torch.float)
        elif torch.is_tensor(s1):
            self.s1 = s1
        else:
            raise TypeError(f"variance is expected to be an array or tensor. "
                            f"Got {type(s1)} instead.")
        if isinstance(s2, ndarray):
            self.s2 = torch.tensor(s2, dtype=torch.float)
        elif torch.is_tensor(s2):
            self.s2 = s2
        else:
            raise TypeError(f"variance is expected to be an array or tensor. "
                            f"Got {type(s2)} instead.")
            
        if isinstance(l1, ndarray):
            self.l1 = torch.tensor(l1, dtype=torch.float)
        elif torch.is_tensor(l):
            self.l1 = l1
        else:
            raise TypeError(f"lengthscale is expected to be an array or tensor. "
                            f"Got {type(l1)} instead.")

        if isinstance(l2, ndarray):
            self.l2 = torch.tensor(l2, dtype=torch.float)
        elif torch.is_tensor(l):
            self.l2 = l2
        else:
            raise TypeError(f"lengthscale is expected to be an array or tensor. "
                            f"Got {type(l2)} instead.")


        if log_transformed:
          self.l1 = torch.exp(self.l1).double()
          self.l2 = torch.exp(self.l2).double()
          self.s1 = torch.exp(self.s1).double()
          self.s2 = torch.exp(self.s2).double()
        super(GenSquaredExponetialKernel, self).__init__(**kwargs)
    
    def forward(self, x1, x2, diag=False, **params):
        global shape
        
        n_features = x1.size()[1]

        first_comp = 1
        second_comp = 0
        for i in range(n_features):
          s = torch.pow(self.l1[i].view(-1,1), 2) + torch.pow(self.l2[i].view(1,-1), 2)
          first_comp *= torch.sqrt(torch.div(2*self.l1[i].view(-1,1)*self.l2[i].view(1,-1), s))
          dist = self.covar_dist(x1[:,i:i+1], x2[:,i:i+1], square_dist=True, diag=diag, **params)
  
          f = dist.div(s)
          second_comp += f   
        
        first_comp *= torch.matmul(self.s1, self.s2.t())
        second_comp = torch.exp(-second_comp)
        k = first_comp * second_comp
        if diag:
            k = k[0]

        return k
    
class MyGPModel(gpytorch.models.ExactGP):
    def __init__(self, s1, s2, l1, l2, train_x, train_y, likelihood):
        super(MyGPModel, self).__init__(train_x, train_y, likelihood)
        self.set_train_data(train_x, train_y)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(GenSquaredExponetialKernel(s1, s2, l1, l2))

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)


In [89]:
likelihood = gpytorch.likelihoods.GaussianLikelihood()

In [90]:

####### Prepare fake data ########
X = torch.randn(1000,4)[:,:-1]
y = X[:,-1]
train_n = int(floor(0.8 * len(X)))
train_x = X[:train_n, :]
train_y = y[:train_n]

test_x = X[train_n:, :]
test_y = y[train_n:]

s1 = np.array([0.1]*3)
s2 = np.array([0.2]*3)
l1 = np.array([3.0]*3)
l2 = np.array([4.0]*3)

model = MyGPModel(s1, s2, l1, l2, train_x=train_x, train_y=train_y, likelihood=likelihood)

# if torch.cuda.is_available():
#     train_x = train_x.cuda()
#     train_y = train_y.cuda()
#     model = model.cuda()
#     likelihood = likelihood.cuda()


In [91]:
ker = GenSquaredExponetialKernel(s1, s2, l1, l2)

In [92]:
ker(train_x)

In [93]:
ker(train_x).diag()

tensor([0.0564, 0.0564, 0.0564, 0.0564, 0.0564, 0.0564, 0.0564, 0.0564, 0.0564,
        0.0564, 0.0564, 0.0564, 0.0564, 0.0564, 0.0564, 0.0564, 0.0564, 0.0564,
        0.0564, 0.0564, 0.0564, 0.0564, 0.0564, 0.0564, 0.0564, 0.0564, 0.0564,
        0.0564, 0.0564, 0.0564, 0.0564, 0.0564, 0.0564, 0.0564, 0.0564, 0.0564,
        0.0564, 0.0564, 0.0564, 0.0564, 0.0564, 0.0564, 0.0564, 0.0564, 0.0564,
        0.0564, 0.0564, 0.0564, 0.0564, 0.0564, 0.0564, 0.0564, 0.0564, 0.0564,
        0.0564, 0.0564, 0.0564, 0.0564, 0.0564, 0.0564, 0.0564, 0.0564, 0.0564,
        0.0564, 0.0564, 0.0564, 0.0564, 0.0564, 0.0564, 0.0564, 0.0564, 0.0564,
        0.0564, 0.0564, 0.0564, 0.0564, 0.0564, 0.0564, 0.0564, 0.0564, 0.0564,
        0.0564, 0.0564, 0.0564, 0.0564, 0.0564, 0.0564, 0.0564, 0.0564, 0.0564,
        0.0564, 0.0564, 0.0564, 0.0564, 0.0564, 0.0564, 0.0564, 0.0564, 0.0564,
        0.0564, 0.0564, 0.0564, 0.0564, 0.0564, 0.0564, 0.0564, 0.0564, 0.0564,
        0.0564, 0.0564, 0.0564, 0.0564, 

In [87]:

################################# Training step #############################
import os
smoke_test = ('CI' in os.environ)
training_iter = 2 if smoke_test else 10
# Find optimal model hyperparameters

model.train()
likelihood.train()
# Use Adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

# Loss func for GP, the log marginal likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
#train_x, train_y = train_x.type(torch.DoubleTensor), train_y.type(torch.DoubleTensor)
for i in range(training_iter):
  # Zero gradient from previous iteration
  optimizer.zero_grad()
  output = model(train_x)

  # Calculate backprop gradients
  loss = -mll(output, train_y)
  loss.backward()

  print('Iter %d/%d - Loss: %.3f' % (i + 1, training_iter, loss.item()))
  optimizer.step()

########################## Make predictions ###############################
# Get into evaluation (predictive posterior) mode
model.eval()
likelihood.eval()

# Make predictions by feeding model through likelihood
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    observed_pred = likelihood(model(test_x))
    mean = observed_pred.mean
    lower, upper = observed_pred.confidence_region()

test_results_gpytorch = np.median((test_y - mean) / test_y, axis=0)
test_results_gpytorch

Iter 1/10 - Loss: 0.503
Iter 2/10 - Loss: 0.455
Iter 3/10 - Loss: 0.408
Iter 4/10 - Loss: 0.361
Iter 5/10 - Loss: 0.314
Iter 6/10 - Loss: 0.266
Iter 7/10 - Loss: 0.219
Iter 8/10 - Loss: 0.172
Iter 9/10 - Loss: 0.125
Iter 10/10 - Loss: 0.078


GaussianLikelihood(
  (noise_covar): HomoskedasticNoise(
    (raw_noise_constraint): GreaterThan(1.000E-04)
  )
)